In [3]:
import sqlite3
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import warnings
warnings.filterwarnings("ignore")

Récupération des données

In [4]:
db_path = '../../db.sqlite3'
conn = sqlite3.connect(db_path)

menu = pd.read_sql_query("SELECT * FROM main_pizza", conn)
orders = pd.read_sql_query("SELECT * FROM main_order", conn)

conn.close()

Préparation des données

In [5]:
order_data = orders.merge(menu, on='pizza_id')

order_data_grouped = order_data.groupby(['client_id', 'name'])['name'].count().unstack(fill_value=0)

# Création des labels pour recommandation
# La pizza la plus commandée pour chaque client devient la cible
order_data_grouped['most_ordered'] = order_data_grouped.idxmax(axis=1)
target = order_data_grouped['most_ordered']
features_encoded = order_data_grouped.drop(columns=['most_ordered'])

# Encodage des pizzas
target_encoded = target.astype('category').cat.codes
pizza_name_mapping = dict(enumerate(target.astype('category').cat.categories))  

Entraînement

In [6]:
# Split
X_train, X_test, y_train, y_test = train_test_split(features_encoded, target_encoded, test_size=0.2, random_state=42)

model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Prédictions
y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print("\nClassification Report:\n", classification_report(y_test, y_pred, target_names=pizza_name_mapping.values()))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))

Accuracy: 0.9264285714285714

Classification Report:
                                             precision    recall  f1-score   support

                The Barbecue Chicken Pizza       0.94      0.98      0.96       122
                        The Big Meat Pizza       0.89      1.00      0.94        72
                      The Brie Carre Pizza       1.00      0.75      0.86        16
                       The Calabrese Pizza       0.96      0.74      0.83        34
              The California Chicken Pizza       0.91      1.00      0.95        89
                 The Chicken Alfredo Pizza       1.00      0.89      0.94        28
                   The Chicken Pesto Pizza       1.00      0.92      0.96        38
                  The Classic Deluxe Pizza       0.96      0.98      0.97       100
                     The Five Cheese Pizza       0.89      0.91      0.90        44
                     The Four Cheese Pizza       0.95      1.00      0.98        59
                     

Exemple de recommandation

In [7]:
import random

# Exemple : recommander une pizza en retirant une commande existante
def recommend_missing_pizza(client_id):
    if client_id not in order_data_grouped.index:
        return f"Client ID {client_id} non trouvé dans les données."
    
    # Historique client
    client_orders = order_data_grouped.loc[client_id].drop('most_ordered').copy()

    # Sélectionner une pizza avec au moins une commande
    pizzas_ordered = client_orders[client_orders > 0]
    if pizzas_ordered.empty:
        return f"Le client {client_id} n'a pas d'historique suffisant pour effectuer une recommandation."
    
    # Retirer une pizza aléatoire de l'historique
    missing_pizza = random.choice(pizzas_ordered.index)
    client_orders[missing_pizza] -= 1

    modified_history = client_orders[client_orders > 0].to_dict()

    formatted_modified_history = "\n".join([f"- {pizza}: {count}" for pizza, count in modified_history.items()])

    # Prédire la pizza manquante
    client_vector_encoded = client_orders.values.reshape(1, -1)
    predicted_pizza_code = model.predict(client_vector_encoded)[0]
    predicted_pizza_name = pizza_name_mapping[predicted_pizza_code]

    return {
        "client_id": client_id,
        "formatted_modified_history": formatted_modified_history,
        "pizza_missing": missing_pizza,
        "predicted_pizza": predicted_pizza_name,
        "is_correct": missing_pizza == predicted_pizza_name
    }

for example_client_id in range(1,20):
    result = recommend_missing_pizza(example_client_id)
    print(f"Client ID : {result['client_id']}")
    print("Historique modifié (sans la pizza manquante) :")
    print(result['formatted_modified_history'])
    print(f"Pizza retirée (manquante) : {result['pizza_missing']}")
    print(f"Pizza recommandée : {result['predicted_pizza']}")
    print(f"Recommandation correcte ? {'Oui' if result['is_correct'] else 'Non'}")
    


Client ID : 1
Historique modifié (sans la pizza manquante) :
- The Classic Deluxe Pizza: 1
- The Greek Pizza: 1
- The Mexicana Pizza: 1
- The Sicilian Pizza: 1
- The Spinach Pesto Pizza: 1
- The Spinach Supreme Pizza: 1
- The Spinach and Feta Pizza: 1
Pizza retirée (manquante) : The Hawaiian Pizza
Pizza recommandée : The Classic Deluxe Pizza
Recommandation correcte ? Non
Client ID : 2
Historique modifié (sans la pizza manquante) :
- The Calabrese Pizza: 1
- The Five Cheese Pizza: 2
- The Greek Pizza: 1
- The Napolitana Pizza: 1
Pizza retirée (manquante) : The Greek Pizza
Pizza recommandée : The Five Cheese Pizza
Recommandation correcte ? Non
Client ID : 3
Historique modifié (sans la pizza manquante) :
- The California Chicken Pizza: 3
- The Green Garden Pizza: 1
- The Mexicana Pizza: 2
- The Thai Chicken Pizza: 1
Pizza retirée (manquante) : The Pepperoni Pizza
Pizza recommandée : The California Chicken Pizza
Recommandation correcte ? Non
Client ID : 4
Historique modifié (sans la pizza 